<h1 align=center><font size = 6>Image Classification using Convolutional Neural Network</font></h1>


---




<h3><strong>Done by</strong></h3>
<p>Firda Anindita Latifah</p>
<p>firdaaninditalatifah@gmail.com</p>

In [ ]:
#Load dataset rockpaperscissors from google drive

from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#Changing the working directory

%cd /content/gdrive/My Drive/Data

In [ ]:
#Extract from zip and define the directory

import zipfile,os

local_zip = '/tmp/rockpaperscissors.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/rockpaperscissors/rps-cv-images'
os.listdir(base_dir)

In [ ]:
#Image augmentation and split data into training and validation (6/4)

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
  rescale=1./255,
  rotation_range=20,
  horizontal_flip=True,
  shear_range = 0.2,
  fill_mode = 'nearest',
  validation_split=0.4)

In [ ]:
#Prepare the training and validation dataset

train_generator = train_datagen.flow_from_directory(
  base_dir,
  target_size=(150, 150),  
  batch_size=32,
  class_mode='categorical',
  subset='training')

validation_generator = train_datagen.flow_from_directory(
  base_dir, 
  target_size=(150, 150), 
  batch_size=32, 
  class_mode='categorical',
  subset='validation')

In [ ]:
#Define CNN Model

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense

model = Sequential([
  Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
  MaxPooling2D(2, 2),
  Conv2D(64, (3,3), activation='relu'),
  MaxPooling2D(2,2),
  Conv2D(128, (3,3), activation='relu'),
  MaxPooling2D(2,2),
  Conv2D(512, (3,3), activation='relu'),
  MaxPooling2D(2,2),
  Flatten(),
  Dropout(0.5),
  Dense(512, activation='relu'),
  Dense(3, activation='softmax')
])

In [ ]:
#Define callback function to stop the epoch after the accuracy is greater than 96%

class Call(tf.keras.callbacks.Callback): 
  def on_epoch_end(self, epoch, logs={}): 
    if(logs.get('accuracy') > 0.96 and logs.get('val_accuracy') > 0.96):
      print("\nAccuracy > 96%") 
      self.model.stop_training = True 
 
callbacks = Call()

In [ ]:
#Compile model using 'adam' optimizer and 'categorical_crossentropy' loss function

from tensorflow import keras

model.compile(
  optimizer=keras.optimizers.Adam(learning_rate=0.001), 
  loss='categorical_crossentropy', 
  metrics=['accuracy']
)

model.summary()

In [ ]:
#Train the model

history = model.fit(
  train_generator,
  steps_per_epoch=40,
  epochs=15,
  validation_data=validation_generator, 
  validation_steps=25,
  verbose=2,
  callbacks=[callbacks]
)

In [ ]:
#Accuration and loss plot

from matplotlib import pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy'] 
loss = history.history['loss'] 
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy') 
plt.title('Training and validation accuracy') 
plt.legend()

plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.ylim((0,1))
plt.figure()

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss') 
plt.title('Training and validation loss')
plt.legend()

plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.ylim((0,1))

plt.show()

In [ ]:
#Predict the new data

import numpy as np
from google.colab import files
from keras.preprocessing import image
import matplotlib.image as mpimg
%matplotlib inline

uploaded = files.upload()

for test in uploaded.keys():
  path = test
  img = image.load_img(path, target_size=(150,150))
  imgplot = plt.imshow(img)
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  output_class = np.argmax(classes)

  print(test)
  if output_class==0:
    print('This is a paper')
  elif output_class==1:
    print('This is a rock')
  else:
    print('This is a scissors')